In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tslearn.metrics import dtw
from scipy.cluster.hierarchy import dendrogram, linkage

# Célula 1: Carregar os Dados

In [4]:
file_path = pd.read_csv('/home/mauricio/Área de Trabalho/BASE PARA MATRIZ - SOUTH.csv')
file_path

file_path.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22632 entries, 0 to 22631
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SEMANA              22632 non-null  int64  
 1   Ano                 22632 non-null  int64  
 2   Artigo              22597 non-null  object 
 3   Linha               22586 non-null  object 
 4   Material            22233 non-null  object 
 5   Venda               22632 non-null  float64
 6   QtdDistintaDeLojas  22632 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.2+ MB


In [3]:
print(file_path.isnull().sum())
nulos = file_path.isnull().sum()
if nulos.sum() > 0:
  nulos.dropna(inplace=True)
  print(f"Tem dados nulos, tem {nulos} linhas nulas ")
else:
  print('Não tem dados nulos')

SEMANA                  0
Ano                     0
Artigo                 35
Linha                  46
Material              399
Venda                   0
QtdDistintaDeLojas      0
dtype: int64
Tem dados nulos, tem SEMANA                  0
Ano                     0
Artigo                 35
Linha                  46
Material              399
Venda                   0
QtdDistintaDeLojas      0
dtype: int64 linhas nulas 


In [5]:
file_path.dropna(subset=['Artigo', 'Linha', 'Material'], how='all', inplace=True)  # Remover linhas onde todas as três colunas são nulas
file_path.replace({'/': '.'}, regex=True, inplace=True)         # Substituir '/' por '.'
file_path = file_path.replace(r'^\s*$', np.nan, regex=True)     # Substituir strings vazias por NaN
file_path.dropna(inplace=True)                                  # Remover valores nulos
file_path.reset_index(drop=True, inplace=True)                  # Resetar índice após remoção de valores nulos


file_path

,SEMANA,Ano,Artigo,Linha,Material,Venda,QtdDistintaDeLojas
0,37,2024,BERMUDA,2 QUALIDADE,SALDO,3.0,4
1,49,2024,CASACO,MASCULINA,MLH MD DIF,0.0,99
2,20,2024,SAIDA DE PRAIA,FEMININA,TEC PL.SAR MD DIF,0.0,3
3,47,2024,REGATA,MASCULINA,MLH BAS,2931.0,99
4,43,2024,BERMUDA,UNISSEX,VARIADO,0.0,1
...,...,...,...,...,...,...,...
22228,8,2024,T-SHIRT,MASCULINA,MLH MD DIF MG LG,4.0,65
22229,44,2024,CAMISA,MASCULINA,LINHO,1239.0,99
22230,32,2024,BIQUINI,2 QUALIDADE,VARIADO,0.0,6
22231,5,2024,VARIADO,FEMININA,VARIADO,0.0,1


In [26]:
#Quais são os produtos mais vendidos por categoria?
df_sum = file_path.groupby(["Artigo", "Linha", "Material"])["Venda"].sum().reset_index()
df_sum = file_path.sort_values(by=["Venda"], ascending=[True])  # Ordena por categoria e quantidade
df_sum
df_moresales = file_path.loc[df_sum.groupby("Venda")["Venda"].idxmax()]
display(df_moresales)

,SEMANA,Ano,Artigo,Linha,Material,Venda,QtdDistintaDeLojas
6271,52,2024,T-SHIRT,MASCULINA,MLH BAS MG LG,-6.0,99
18438,49,2024,T-SHIRT,MASCULINA,MLH MD DIF MG LG,-4.0,99
4397,50,2024,T-SHIRT,MASCULINA,MLH MD DIF MG LG,-3.0,99
18775,52,2024,BLUSA,INFANTIL FEMININA,TEC PL MD DIF,-2.0,99
12782,48,2024,T-SHIRT,MASCULINA,MLH BAS MG LG,-1.0,99
...,...,...,...,...,...,...,...
22009,52,2023,T-SHIRT,MASCULINA,MLH MD DIF,47948.0,97
2733,50,2023,T-SHIRT,MASCULINA,MLH MD DIF,52838.0,96
20226,51,2024,T-SHIRT,MASCULINA,MLH MD DIF,74444.0,99
19784,52,2024,T-SHIRT,MASCULINA,MLH MD DIF,77667.0,99


In [40]:
scaler = StandardScaler()
file_path['normalized_sales'] = scaler.fit_transform(file_path[['Venda']])
scaler

StandardScaler()

In [39]:
# Célula 3: Criar uma Matriz de Similaridade usando Correlação de Pearson
similarity_matrix = file_path.pivot(columns='SEMANA', values='Venda').corr()
similarity_matrix

SEMANA,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
SEMANA,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Célula 4: Aplicar K-Means para Clusterização
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
file_path['cluster'] = kmeans.fit_predict(similarity_matrix.fillna(0))

ValueError: Length of values (53) does not match length of index (22233)

In [42]:
# Célula 5: Avaliação dos Clusters com Silhouette Score
silhouette_avg = silhouette_score(similarity_matrix.fillna(0), file_path['cluster'])
print(f'Silhouette Score: {silhouette_avg}')

KeyError: 'cluster'

In [ ]:
# Célula 6: Visualização dos Clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=file_path['week'], y=file_path['normalized_sales'], hue=file_path['cluster'], palette='viridis')
plt.title('Clusters de Produtos com Perfis de Vendas Semelhantes')
plt.show()

In [ ]:
# Célula 7: Clusterização Hierárquica para Melhor Visualização
linkage_matrix = linkage(similarity_matrix.fillna(0), method='ward')
plt.figure(figsize=(12, 6))
dendrogram(linkage_matrix)
plt.title('Dendrograma da Clusterização Hierárquica')
plt.show()